In [1]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/usa-los-angeles-crimes-data-2020-to-2023/Crime_Data_from_2020_to_Present.csv',
                index_col=['DR_NO'], parse_dates=['Date Rptd', 'DATE OCC'])
df.head()

/tmp/ipykernel_20/1435733065.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(filepath_or_buffer='/kaggle/input/usa-los-angeles-crimes-data-2020-to-2023/Crime_Data_from_2020_to_Present.csv',
/tmp/ipykernel_20/1435733065.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(filepath_or_buffer='/kaggle/input/usa-los-angeles-crimes-data-2020-to-2023/Crime_Data_from_2020_to_Present.csv',


,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
DR_NO,,,,,,,,,,,,,,,,,,,,,
10304468,2020-01-08,2020-01-08,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,0444 0913,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
190101086,2020-01-02,2020-01-01,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,0416 1822 1414,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
200110444,2020-04-14,2020-02-13,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,1501,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
191501505,2020-01-01,2020-01-01,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 1402,...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
191921269,2020-01-01,2020-01-01,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [2]:
from plotly.express import bar
top_twenty_by_count_df = df[['Crm Cd Desc']].groupby(by=['Crm Cd Desc']).size().reset_index().sort_values(ascending=False, by=0).head(n=20)
bar(data_frame=top_twenty_by_count_df, x='Crm Cd Desc', y=0)

Most of our top twenty crimes by code by count are property crimes. 

In [3]:
from plotly.express import line
top_twenty = top_twenty_by_count_df['Crm Cd Desc'].values
counts_df = df[['DATE OCC', 'Crm Cd Desc']].set_index('DATE OCC').groupby(by=['Crm Cd Desc']).resample('M').size().reset_index()
lines = line(data_frame=counts_df[counts_df['Crm Cd Desc'].isin(top_twenty)], x='DATE OCC', y=0, color='Crm Cd Desc', log_y=False, height=900, facet_col='Crm Cd Desc', facet_col_wrap=4)
# our subplot labels are redundant so let's remove them
for label in lines['layout']['annotations']:
    label['text'] = ''
lines.show()

We needed to roll these up into monthly means to damp what we hope is reporting noise rather than crime surges that happen on the first of the month.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 829778 entries, 10304468 to 230906458
Data columns (total 27 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date Rptd       829778 non-null  datetime64[ns]
 1   DATE OCC        829778 non-null  datetime64[ns]
 2   TIME OCC        829778 non-null  int64         
 3   AREA            829778 non-null  int64         
 4   AREA NAME       829778 non-null  object        
 5   Rpt Dist No     829778 non-null  int64         
 6   Part 1-2        829778 non-null  int64         
 7   Crm Cd          829778 non-null  int64         
 8   Crm Cd Desc     829778 non-null  object        
 9   Mocodes         714922 non-null  object        
 10  Vict Age        829778 non-null  int64         
 11  Vict Sex        720479 non-null  object        
 12  Vict Descent    720471 non-null  object        
 13  Premis Cd       829768 non-null  float64       
 14  Premis Desc     829286 non-null

In [5]:
df.nunique()

Date Rptd           1399
DATE OCC            1399
TIME OCC            1439
AREA                  21
AREA NAME             21
Rpt Dist No         1206
Part 1-2               2
Crm Cd               138
Crm Cd Desc          138
Mocodes           276670
Vict Age             103
Vict Sex               5
Vict Descent          20
Premis Cd            313
Premis Desc          306
Weapon Used Cd        79
Weapon Desc           79
Status                 6
Status Desc            6
Crm Cd 1             140
Crm Cd 2             123
Crm Cd 3              37
Crm Cd 4               6
LOCATION           63880
Cross Street        9731
LAT                 5402
LON                 4970
dtype: int64

In [6]:
from plotly.express import histogram
histogram(data_frame=df[df['Vict Sex'].isin({'F', 'M'})], x='Vict Age', facet_col='Vict Sex')

In [7]:
histogram(data_frame=df, x='AREA')